In [1]:
import psycopg2
import sys
import pprint
from datetime import date, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import os,sys
import matplotlib.patches as mpatches
from sklearn import preprocessing

#sys.path.append("../Mfig/")
#import mplp
from itertools import chain
import numpy as np
import csv
from collections import defaultdict
from datetime import date, timedelta
from datetime import datetime
from scipy.stats import entropy
%matplotlib inline

## DB connection

In [2]:
def conectionDB():
    conn_string = "dbname='stic' port='5432' user='stic' password='stic2019' host='gpmaster.as-dell.copernic.local'"
    #print ("Connecting to database ->{}".format(conn_string))
    conn = psycopg2.connect(conn_string)
    return conn

# build dictionaries


In [3]:
dict_dpto = dict()
dict_province = dict()
dict_district = dict()
with open('../data/ubigeo_inei.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        #print (row)
        if (str(row[0])!=''):
            dict_dpto[str(row[3]).lower()]=(str(row[0]))
        if (str(row[1])!=''):
            dict_province[(row[4]).lower()]=(row[0]+''+row[1])
        if (str(row[2])!=''):
            dict_district[(row[5]).lower()]=(row[0]+''+row[1]+''+row[2])
        
dict_gender = {'M':1,'F':0,'':-1}
dict_card = {'TD':1,'TC':0,'':-1}
dict_region = {'NORTE':0,'ORIENTE':1,'SUR':2,'CENTRO':3,'SIERRA CENTRAL':4,
               'LIMA CENTRO':5,'LIMA ESTE':6,'LIMA SUR':7,'LIMA NORTE':8,
               'LIMA MODERNA':9,'LIMA PROVINCIA':10,'CALLAO':11,'':12}


In [4]:
columnNames=[ 
        'social_class' ,     #-- 0
        'country_code',      #-- 1
        'amount_usd',        #-- 2 *
        'client_age',        #-- 3 *
        'client_gender',     #-- 4
        'debit_type',        #-- 5 Credit or debit card
        'agency_region',     #-- 9
        'merchant_departement', #-- 10
        ]


## extraction loop

In [5]:
def extract_all (index):
    PLIMIT = 1000000
    POFFSET = 1000000
    #NUM = 93 # Millions of records (92 470 071)
    NUM = 2 #for test 
    i=index
    while (i < NUM):
        print ("Processing {}/{} ...".format(i,NUM))
        query ="""
            SELECT 
                social_class,      -- 0
                amount_usd,        -- 1 *
                client_age,        -- 2 *
                client_gender,     -- 3
                debit_type,        -- 4 Credit or debit card
                agency_region,     -- 5
                merchant_departement -- 6
            FROM 
                public.bbva 
            INNER JOIN 
                client
            ON 
                client.client_id = bbva.client_id
            WHERE
                bbva.merchant_id != '00000000'
            ORDER BY 
                bbva.date ASC
            LIMIT {}
            OFFSET {}
        ;
        """.format(PLIMIT,POFFSET*i)
        conn = conectionDB()
        cursor = conn.cursor();
        df = pd.read_sql(query, conn)
        df.fillna(-1, inplace=True)
        
        df['small_social_class'] = df.apply(lambda row: 1 if (row.social_class<=3) else 0 , axis=1)
        del df['social_class']
      
        df=df.replace({ "merchant_departement":dict_dpto,
                        "agency_region": dict_region
                         })
        df.fillna(-1, inplace=True)
        ###############################        
        #  Normalize numeric columns  #
        ###############################
        min_max_scaler = preprocessing.MinMaxScaler()
        
        # Create x, where x the 'scores' column's values as floats
        x_amount_usd = df[['amount_usd']].values.astype(float)
        x_client_age = df[['client_age']].values.astype(float)

        # Create an object to transform the data to fit minmax processor
        x_scaled_amount_usd = min_max_scaler.fit_transform(x_amount_usd)
        x_scaled_client_age = min_max_scaler.fit_transform(x_client_age)

        # Run the normalizer on the dataframe
        df_normalized_amount_usd = pd.DataFrame(x_scaled_amount_usd)
        df_normalized_client_age = pd.DataFrame(x_scaled_client_age)
        
        #######################################        
        #  One hot enconding numeric columns  #
        #######################################
        
        df_client_gender_henc = pd.get_dummies(df['client_gender'], prefix = 'client_gender_')
        df_debit_type_henc = pd.get_dummies(df['debit_type'], prefix = 'debit_type_')
        df_agency_region_henc = pd.get_dummies(df['agency_region'], prefix = 'agency_region_')
        df_merchant_departement_henc = pd.get_dummies(df['merchant_departement'], prefix = 'merchant_departement_')
        
        newdf = pd.concat([df_normalized_amount_usd, 
                   df_normalized_client_age,
                   df_client_gender_henc,
                   df_debit_type_henc,
                   df_agency_region_henc,
                   df_merchant_departement_henc
                  ], axis=1)
        newdf['social_class'] = df['small_social_class']
        
        newdf = newdf[newdf['merchant_departement__99'] != 1]
        newdf = newdf[newdf['client_gender__'] != 1]
        newdf = newdf[newdf['agency_region__12'] != 1]
        
        del newdf['merchant_departement__99']
        del newdf['client_gender__']
        del newdf['agency_region__12']
        
        print (newdf.columns)

        newdf.to_csv (r'../data_output/export_dataframe_{}v5.csv'.format(i), index = None, header=False)
        i+=1

In [6]:
extract_all(0)

Processing 0/2 ...
Index([                                    0,
                                           0,
                          'client_gender__F',
                          'client_gender__M',
                            'debit_type__TC',
                            'debit_type__TD',
                          'agency_region__0',
                          'agency_region__1',
                          'agency_region__2',
                          'agency_region__3',
                          'agency_region__4',
                          'agency_region__5',
                          'agency_region__6',
                          'agency_region__7',
                          'agency_region__8',
                          'agency_region__9',
                         'agency_region__10',
                         'agency_region__11',
            'merchant_departement__AMAZONAS',
              'merchant_departement__ANCASH',
            'merchant_departement__APURIMAC',
            'me